In [1]:
import numpy as np
import sympy as sp
from control import matlab
from control import canonical_form

In [2]:
A = np.array([[-3, 4],[-1, 5]])
B = np.array([[1],[0]])
C = np.array([[0,2]])
D = np.array([[0]])
Pss = matlab.ss(A, B, C, D)

In [3]:
Pr, T = canonical_form(Pss, form = 'reachable') #可制御正準系

In [4]:
print(Pr)

A = [[ 2. 11.]
     [ 1.  0.]]

B = [[1.]
     [0.]]

C = [[-0. -2.]]

D = [[0.]]



In [8]:
P1 = matlab.ss2tf(Pss) #伝達関数に変換
P2 = matlab.ss2tf(Pr)
print(P1, P2)


-8.882e-16 s - 2
----------------
 s^2 - 2 s - 11
 
4.441e-16 s - 2
---------------
s^2 - 2 s - 11



In [48]:
A = np.array([[-3, 8], [-3, 7]]) #対角正準系
Pss = matlab.ss(A, B, C, D)
Pm, T = canonical_form(Pss, form = 'modal')

In [49]:
print(Pm)

A = [[3.00000000e+00 7.63648021e-16]
     [5.79790797e-16 1.00000000e+00]]

B = [[ 2.5       ]
     [-3.35410197]]

C = [[-1.2        -0.89442719]]

D = [[0.]]



In [59]:
A = np.array([[1, -9], [1, -5]]) #対角化不可の場合も無理やり計算してしまう　
Pss = matlab.ss(A, B, C, D)
Pm, T = canonical_form(Pss, form = 'modal')
print(Pm)

A = [[-2.00000000e+00  4.65288609e-08]
     [-2.48143998e-08 -2.00000000e+00]]

B = [[1.05409255e+00]
     [6.79637885e+07]]

C = [[ 6.32455532e-01 -9.80914516e-09]]

D = [[0.]]



In [56]:
A = np.array([[-3, 8], [-3, 7]])
sp.init_printing()
A = sp.eye(2)*A

In [57]:
A.eigenvects() #固有ベクトル

⎡⎛      ⎡⎡2⎤⎤⎞  ⎛      ⎡⎡4/3⎤⎤⎞⎤
⎢⎜1, 1, ⎢⎢ ⎥⎥⎟, ⎜3, 1, ⎢⎢   ⎥⎥⎟⎥
⎣⎝      ⎣⎣1⎦⎦⎠  ⎝      ⎣⎣ 1 ⎦⎦⎠⎦

In [55]:
A.diagonalize() #対角化

⎛⎡2  4⎤  ⎡1  0⎤⎞
⎜⎢    ⎥, ⎢    ⎥⎟
⎝⎣1  3⎦  ⎣0  3⎦⎠

In [61]:
A = np.array([[1, -9], [1, -5]])  #ジョルダン
A = sp.eye(2)*A
A.jordan_form()

⎛⎡3  1⎤  ⎡-2  1 ⎤⎞
⎜⎢    ⎥, ⎢      ⎥⎟
⎝⎣1  0⎦  ⎣0   -2⎦⎠